<a href="https://colab.research.google.com/github/MorningStarTM/behaviour-cloning/blob/main/notebook/MLP_BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    patchelf \
    xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
The following additional packages will be installed:
  libegl-dev libfontenc1 libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev
  libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6 libxfont2 libxkbfile1
  x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils xserver-common
The following NEW packages will be installed:
  libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev libxfont2 libxkbfile1 patchelf x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 25 newly installed, 0 to remove and 45 not upgraded.
Need to get 11.9 MB of archives.
After this operation, 31.5 MB of additional disk space will b

In [2]:
!pip install gym==0.21.0
!pip install free-mujoco-py mujoco
!pip install transformers
!pip install datasets
!pip install imageio-ffmpeg

!pip install colabgymrender==1.0.2
!pip install xvfbwrapper
!pip install imageio==2.4.1
!pip install imageio-ffmpeg
!pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found e

In [3]:
!git clone https://github.com/MorningStarTM/behaviour-cloning.git

Cloning into 'behaviour-cloning'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 27 (delta 5), reused 22 (delta 4), pack-reused 0
Receiving objects: 100% (27/27), 5.65 KiB | 5.65 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [4]:
%cd behaviour-cloning

/content/behaviour-cloning


In [5]:
!ls

algorithms  LICENSE  README.md	utils


In [6]:
import os
import random
from tqdm import tqdm

from dataclasses import dataclass
import gym
import numpy as np
import torch
import torch.nn as nn

from datasets import load_dataset
from algorithms import MLPBehaviorCloning
from utils import DataLoader

In [7]:
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "walker2d-expert-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
dl = DataLoader(dataset["train"])

In [9]:
dl(2)

{'states': tensor([[-0.4778,  0.3098,  0.6273,  0.7368,  0.8196,  0.4955, -0.6146, -0.8659,
           0.6099,  1.1138,  0.2474,  0.0468,  0.6430, -0.0119,  0.0734, -0.5614,
          -1.6186],
         [ 1.6439,  0.3610, -0.4334, -1.2027, -1.4005,  0.5669,  1.0918,  0.7811,
           0.1263, -0.2363, -0.9191, -1.4280,  0.6255,  0.2000,  0.1599, -0.0244,
          -0.0094]]),
 'actions': tensor([[ 0.9471, -0.4040,  0.9464,  0.8634, -0.0539, -0.9894],
         [ 0.7584,  0.2864, -0.3584,  0.6145,  0.7163,  0.9825]])}

In [10]:
env = gym.make("Walker2d-v2")

state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Walker2d-v2 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/Cython/Distutils/old_build_ext.py:15: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group


Compiling /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [11]:
model = MLPBehaviorCloning(state_size, action_size, 256, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [12]:
epochs = 2000
batch_size = 64
num_iters = (len(dataset["train"])-1) // 64 + 1


pbar = tqdm(range(epochs))

for e in pbar:
    sum_loss = 0.0
    for i in range(num_iters):
        data = dl(batch_size)
        loss = model.forward_loss(**data)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()

        sum_loss += loss.item()

    pbar.set_postfix(loss=sum_loss)

100%|██████████| 2000/2000 [02:22<00:00, 14.02it/s, loss=0.586]


In [13]:
import mujoco_py
import gym

from colabgymrender.recorder import Recorder

  np.bool8: (False, True),

  from scipy.ndimage.filters import sobel



In [18]:
env = gym.make("Walker2d-v2")
env = Recorder(env, ".", fps=30)

device = "cpu"
max_ep_len = 100000
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]

  logger.warn(

  logger.warn(

  deprecation(

  deprecation(

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(



In [19]:
state = env.reset()

for t in range(max_ep_len):
    with torch.no_grad():
        state = torch.from_numpy(state).reshape(1, state.shape[0]).to(device=device, dtype=torch.float32)
        action = model(state)[0].numpy()

    state, reward, done, _ = env.step(action)

    if done:
        break

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(

  if not isinstance(terminated, (bool, np.bool8)):



In [20]:
env.play()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [17]:
!ffmpeg -i "replay" -vcodec h264 replay.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab